19.Nguyễn Xuân Hội-22174600040

Bài 1:Hãy viết câu lệnh SQLđể tính sự tương quan giữa Avà B theo công thức:

CREATE TABLE data1 (
	id INT PRIMARY KEY,
    A INT,
	B INT
);
INSERT INTO data1 (id, A, B) VALUES
(1,2,8),
(2,5,6),
(3,7,5),
(4,9,6),
(5,5,3),
(6,1,2),
(7,4,2);
SELECT 
    (n * sum_ab - sum_a * sum_b) /
    (SQRT(n * sum_a2 - POWER(sum_a, 2)) * SQRT(n * sum_b2 - POWER(sum_b, 2))) AS r_ab
FROM (
    SELECT 
        COUNT(*) AS n,
        SUM(A) AS sum_a,
        SUM(B) AS sum_b,
        SUM(A * B) AS sum_ab,
        SUM(A * A) AS sum_a2,
        SUM(B * B) AS sum_b2
    FROM data1
) AS stats;


kết quả : 0.240874001161144

In [1]:
import sqlite3
import math

# Khởi tạo SQLite trong RAM
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng và chèn dữ liệu
cursor.execute("""
CREATE TABLE data1 (
    id INTEGER PRIMARY KEY,
    A INTEGER,
    B INTEGER
);
""")

rows = [
    (1, 2, 8),
    (2, 5, 6),
    (3, 7, 5),
    (4, 9, 6),
    (5, 5, 3),
    (6, 1, 2),
    (7, 4, 2),
]

cursor.executemany("INSERT INTO data1 (id, A, B) VALUES (?, ?, ?);", rows)

# Truy vấn tính toán các thành phần Pearson
cursor.execute("""
SELECT 
    COUNT(*) AS n,
    SUM(A) AS sum_a,
    SUM(B) AS sum_b,
    SUM(A * B) AS sum_ab,
    SUM(A * A) AS sum_a2,
    SUM(B * B) AS sum_b2
FROM data1;
""")

# Lấy dữ liệu
result = cursor.fetchone()
n, sum_a, sum_b, sum_ab, sum_a2, sum_b2 = result

# Tính hệ số tương quan Pearson
numerator = n * sum_ab - sum_a * sum_b
denominator = math.sqrt(n * sum_a2 - sum_a ** 2) * math.sqrt(n * sum_b2 - sum_b ** 2)

r_ab = numerator / denominator if denominator != 0 else None

# In kết quả
print(f"Hệ số tương quan r_AB = {r_ab}")

# Đóng kết nối
conn.close()


Hệ số tương quan r_AB = 0.240874001161144


Câu 2: Một công ty oto đang kiểm tra 3 loại mẫu mới A, B và C trong 4 ngày, và chấm điểm theo thang từ 1 đến 10 điểm cho mỗi ngày với bảng sau. Liệu có sự khác biệt đáng kể giữa các mẫu dựa trên điểm số mà chúng nhận được trong 4 ngày thử nghiệm không? Kết quả thử nghiệm phụ thuộc vào ngày hay phụ thuộc vào mẫu xe? Hãy chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2.

In [4]:
import sqlite3
import pandas as pd
from scipy.stats import chi2_contingency

# Kết nối SQLite
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Tạo bảng
cur.execute("""
    CREATE TABLE oto (
        day TEXT PRIMARY KEY,
        A FLOAT,
        B FLOAT,
        C iNTEGER
    );
""")

# Chèn dữ liệu
row = [
    ("day1", 8, 9, 7),
    ("day2", 7.5, 8.5, 7),
    ("day3", 6, 7, 8),
    ("day4", 7, 6, 5)
   
]
cur.executemany('INSERT INTO oto VALUES (?, ?, ?, ?)', row)
conn.commit()

# Chuyển sang dạng quan hệ và phân loại điểm (Low / Medium / High)
query = """
    SELECT 
        day,
        model,
        score,
        CASE 
            WHEN score <= 5 THEN 'Low'
            WHEN score < 8 THEN 'Medium'
            ELSE 'High'
        END AS category
    FROM (
        SELECT day, 'A' AS model, A AS score FROM oto
        UNION ALL
        SELECT day, 'B' AS model, B AS score FROM oto
        UNION ALL
        SELECT day, 'C' AS model, C AS score FROM oto
    )
"""
df = pd.read_sql_query(query, conn)
# Tạo bảng chéo giữa model và category
table_model = pd.crosstab(df['model'], df['category'])

# Thực hiện kiểm định chi bình phương
chi2, p, dof, expected = chi2_contingency(table_model)

print("=== Kiểm định theo model ===")
print(table_model)
print(f"Chi2 = {chi2:.4f}, p-value = {p:.4f}")
# Tạo bảng chéo giữa day và category
table_day = pd.crosstab(df['day'], df['category'])

# Thực hiện kiểm định chi bình phương
chi2_day, p_day, dof_day, expected_day = chi2_contingency(table_day)

print("\n=== Kiểm định theo ngày ===")
print(table_day)
print(f"Chi2 = {chi2_day:.4f}, p-value = {p_day:.4f}")


=== Kiểm định theo model ===
category  High  Low  Medium
model                      
A            1    0       3
B            2    0       2
C            1    1       2
Chi2 = 2.7857, p-value = 0.5943

=== Kiểm định theo ngày ===
category  High  Low  Medium
day                        
day1         2    0       1
day2         1    0       2
day3         1    0       2
day4         0    1       2
Chi2 = 5.4286, p-value = 0.4901


Về model:
p-value > 0.05, không có đủ bằng chứng để bác bỏ giả thuyết H0 (không có sự khác biệt đáng kể giữa các mẫu xe).
Theo Ngày:
p-value > 0.05, tiếp tục giữ nguyên giả thuyết H0.

 Câu 3: Bảng flights(departure_time,…) chứa các giá trị thời gian dưới dạng số nguyên (ví dụ: 830 cho 8:30 AM, 1445 cho 2:45 PM). Hãy chuyển đổi các giá trị này thành định dạng thời gian.

In [8]:
import sqlite3

# Kết nối CSDL trong bộ nhớ (hoặc dùng file)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng và dữ liệu mẫu
cursor.execute("""
CREATE TABLE flights (
    id INTEGER,
    departure_time INTEGER
)
""")

# Thêm dữ liệu
flights_data = [
    (1, 830),
    (2, 1445),
    (3, 45),
    (4, 1230),
    (5, 0),
    (6, 30)
]
cursor.executemany("INSERT INTO flights (id, departure_time) VALUES (?, ?)", flights_data)

# Truy vấn chuyển đổi thời gian sang định dạng 12 giờ có AM/PM
cursor.execute("""
SELECT 
    id,
    departure_time,
    printf('%02d:%02d %s',
        CASE 
            WHEN (departure_time / 100) % 12 = 0 THEN 12
            ELSE (departure_time / 100) % 12
        END,
        departure_time % 100,
        CASE 
            WHEN departure_time / 100 < 12 THEN 'AM'
            ELSE 'PM'
        END
    ) AS formatted_time_12h
FROM flights
""")

# In kết quả
for row in cursor.fetchall():
    print(row)

conn.close()


(1, 830, '08:30 AM')
(2, 1445, '02:45 PM')
(3, 45, '12:45 AM')
(4, 1230, '12:30 PM')
(5, 0, '12:00 AM')
(6, 30, '12:30 AM')


Nhận xét:
(departure_time / 100) % 12: chuyển sang định dạng 12 giờ.
Nếu giờ là 0 hoặc 12, thì hiển thị là 12.
Phần AM/PM dựa vào departure_time / 100 < 12.

 Câu 4: Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá trị ngoại lệ lớn hơn 1,5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa.

In [9]:
import sqlite3
import pandas as pd

# Tạo SQLite trong bộ nhớ
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng dữ liệu
cursor.execute('''
CREATE TABLE sales (
    id INTEGER PRIMARY KEY ,
    amount REAL
)
''')

# Thêm dữ liệu mẫu (bao gồm giá trị ngoại lệ)
data = [100, 105, 102, 110, 108, 5000, 107, 109, 103, 106, 104, 6000]
cursor.executemany("INSERT INTO sales (amount) VALUES (?)", [(x,) for x in data])
conn.commit()

# Truy vấn SQL để tính median, MAD và lọc ngoại lệ
query = """
WITH ordered AS (
  SELECT amount,
         ROW_NUMBER() OVER (ORDER BY amount) AS rn,
         COUNT(*) OVER () AS cnt
  FROM sales
),
median_cte AS (
  SELECT AVG(amount) AS median
  FROM ordered
  WHERE rn IN ((cnt + 1) / 2, (cnt + 2) / 2)
),
deviation AS (
  SELECT s.id, s.amount, 
         ABS(s.amount - m.median) AS abs_dev,
         m.median
  FROM sales s
  JOIN median_cte m
),
ordered_dev AS (
  SELECT abs_dev,
         ROW_NUMBER() OVER (ORDER BY abs_dev) AS rn,
         COUNT(*) OVER () AS cnt
  FROM deviation
),
mad_cte AS (
  SELECT AVG(abs_dev) AS mad
  FROM ordered_dev
  WHERE rn IN ((cnt + 1) / 2, (cnt + 2) / 2)
)
SELECT d.id, d.amount, d.median, a.mad,
       ROUND(ABS(d.amount - d.median) / NULLIF(a.mad, 0), 2) AS modified_z
FROM deviation d
JOIN mad_cte a
WHERE ABS(d.amount - d.median) / NULLIF(a.mad, 0) > 1.5
"""

# Thực thi và lấy kết quả
df_outliers = pd.read_sql_query(query, conn)

# Hiển thị kết quả
print(" Các giá trị ngoại lệ (modified_z > 1.5):")
print(df_outliers)


 Các giá trị ngoại lệ (modified_z > 1.5):
   id  amount  median  mad  modified_z
0   1   100.0   106.5  3.0        2.17
1   6  5000.0   106.5  3.0     1631.17
2  12  6000.0   106.5  3.0     1964.50


Bài5:

In [12]:
import sqlite3

# Kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng
cursor.execute("""
CREATE TABLE Patient (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    last_name TEXT,
    weight REAL,
    height REAL
)
""")

# Thêm dữ liệu mẫu
patients = [
    ('A', 60, 165),
    ('D', 55, 163),
    ('C',   55, 160),
    ('A',     60, 155),
    ('D',   55, 162),
]

cursor.executemany("INSERT INTO Patient (last_name, weight, height) VALUES (?, ?, ?)", patients)

# Truy vấn xác định các cặp có thể là một người
cursor.execute("""
SELECT 
    A.id AS id1, A.last_name AS last_name1, A.weight AS weight1, A.height AS height1,
    B.id AS id2, B.last_name AS last_name2, B.weight AS weight2, B.height AS height2,
    CASE 
        WHEN A.last_name = B.last_name AND A.weight = B.weight THEN 'Same person'
        ELSE 'Different person'
    END AS match_result
FROM 
    Patient A
JOIN 
    Patient B 
ON 
    A.id < B.id
""")

# In kết quả
results = cursor.fetchall()
for row in results:
    print(row)

# Đóng kết nối
conn.close()


(1, 'A', 60.0, 165.0, 2, 'D', 55.0, 163.0, 'Different person')
(1, 'A', 60.0, 165.0, 3, 'C', 55.0, 160.0, 'Different person')
(1, 'A', 60.0, 165.0, 4, 'A', 60.0, 155.0, 'Same person')
(1, 'A', 60.0, 165.0, 5, 'D', 55.0, 162.0, 'Different person')
(2, 'D', 55.0, 163.0, 3, 'C', 55.0, 160.0, 'Different person')
(2, 'D', 55.0, 163.0, 4, 'A', 60.0, 155.0, 'Different person')
(2, 'D', 55.0, 163.0, 5, 'D', 55.0, 162.0, 'Same person')
(3, 'C', 55.0, 160.0, 4, 'A', 60.0, 155.0, 'Different person')
(3, 'C', 55.0, 160.0, 5, 'D', 55.0, 162.0, 'Different person')
(4, 'A', 60.0, 155.0, 5, 'D', 55.0, 162.0, 'Different person')
